In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
%matplotlib inline 
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import callbacks
import os
import cv2
import string
import numpy as np

#Init main values
# symbols = string.ascii_lowercase + "0123456789" # All symbols captcha can contain
symbols = "0123456789" # All symbols captcha can contain

num_symbols = len(symbols)
img_shape = (42, 152, 1)

In [3]:
print(num_symbols)

In [4]:
def create_model():
    img = layers.Input(shape=img_shape) # Get image as an input and process it through some Convs
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 25x7
    
    # Get flattened vector and make 5 branches from it. Each branch will predict one letter
    dns = layers.Dense(1024, activation='relu')(mp3)
    flat = layers.Flatten()(dns)

    outs = []
    for _ in range(4):
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='sigmoid')(drop)

        outs.append(res)
    
    # Compile model and return it
    model = Model(img, outs)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

In [5]:
def preprocess_data():
    n_samples = len(os.listdir('../input/d/raminhasani/captcha/test'))
    X = np.zeros((n_samples, 42, 152, 1)) #1070*50*200
    y = np.zeros((4, n_samples, num_symbols)) #5*1070*36

    for i, pic in enumerate(os.listdir('../input/d/raminhasani/captcha/test')):
        # Read image as grayscale
        img = cv2.imread(os.path.join('../input/d/raminhasani/captcha/test', pic), cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4]
        if len(pic_target) < 5:
            # Scale and reshape image
            img = img / 255.0
            img = np.reshape(img, (42, 152, 1))
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((4, num_symbols))
            for j, l in enumerate(pic_target):
                ind = symbols.find(l)
                targs[j, ind] = 1
            X[i] = img
            y[:, i] = targs
    
    # Return final data
    return X, y

X, y = preprocess_data()
X_train, y_train = X[:1700], y[:, :1700]
X_test, y_test = X[1700:], y[:, 1700:]

In [6]:
y.shape

In [7]:
from matplotlib import pyplot as plt
display(display(y[:,0]))
plt.imshow(np.reshape(X[0], (42,152)))

# X_train[0].shape

In [10]:
model=create_model();
model.summary();

In [11]:
#model = create_model()
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3]], batch_size=32, epochs=30,verbose=1, validation_split=0.2)

In [12]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (4, 10))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
    return capt#, sum(probs) / 5



In [13]:
score= model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3]],verbose=1)
print('Test Loss:', score[:5])
print('Test accu:', score[5:])

In [14]:
# Check model on some samples
model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3]])
print(predict('../input/d/raminhasani/captcha/test/1005.jpg'))
print(predict('../input/d/raminhasani/captcha/test/1007.jpg'))
print(predict('../input/d/raminhasani/captcha/test/1011.jpg'))
print(predict('../input/d/raminhasani/captcha/test/1045.jpg'))
print(predict('../input/d/raminhasani/captcha/test/1073.jpg'))

In [15]:
#Lets test an unknown captcha
#preview
%matplotlib inline 
import matplotlib.pyplot as plt
img=cv2.imread('../input/d/raminhasani/captcha/test/8658.jpg',cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap=plt.get_cmap('gray'))

In [16]:
#Lets Predict By Model
print("Predicted Captcha =",predict('../input/d/raminhasani/captcha/test/8658.jpg'))

In [15]:
#testing
#c=0
#for i, pic in enumerate(os.listdir('../input/captcha-version-2-images/samples/samples')):
        # Read image as grayscale
        
        #if i>970:    

            #img = cv2.imread(os.path.join('../input/captcha-version-2-images/samples/samples', pic), cv2.IMREAD_GRAYSCALE)
            #print("Predicted Captcha =",predict(os.path.join('../input/captcha-version-2-images/samples/samples',pic)))
            #plt.imshow(img, cmap=plt.get_cmap('gray'))
            #pr=predict(os.path.join('../input/captcha-version-2-images/samples/samples',pic))
            #pic_target = pic[:-4]
            #if pr==pic_target:
                #c=c+1
                #print(c)
            #print(pic_target)
#print((c/100)*100)